In [10]:
# Trabalho 2 - Realce e Superresolução
# SCC0251 - Image Processing (01/2021)
# Fabiana Dalacqua Mendes - 9894536
# Pedro Henrique Nieuwenhoff - 10377729
import numpy as np
import imageio
import math
import matplotlib.pyplot as plt

In [11]:
def load_images(name_base):
    """
    This function loads and returns a list of 4 images 
    witch name starts with the name base.
    
    Parameters
    ----------
    name_base : string
        The name base of the low resolution images to be loaded
    """
    
    L = []
    for i in range(4):
        filename = "{name_base}{index}.png".format(name_base = name_base, index = i)
        L.append(imageio.imread(filename)) # reading image return uint8 type array

    return L

In [12]:
def enhancement(L,F,gamma):
    """
    This function apply enhancement method in the list of
    low resolution images.
    
    Parameters
    ----------
    L : list
       The list of size 4 containing the low quality images
    F : int
        The enhancement function number that must be between 0 and 3
    gamma : int
        The parameter used in ehancement function number 3
    """
    def histogram(img,no_levels):        
        hist = np.zeros(no_levels).astype(int)
        
        for i in range(no_levels):
            nopix_value_i = np.sum(img == i) # counting number of each intensity level
            hist[i] = nopix_value_i
            
        return hist
            
    def individual_cumulative_histogram(img,no_levels):
        hist = histogram(img,no_levels)
        
        # generating the cumulative histogram
        histC = np.zeros(no_levels).astype(int)
        histC[0] = hist[0]
        for i in range(1,no_levels):
            histC[i] = hist[i] + histC[i-1]
        
        # transfer function
        histT = np.zeros(no_levels).astype(np.uint8)
        
        # equalized image
        N,M = img.shape
        imgEq = np.zeros([N,M]).astype(np.uint8)
        
        # computing equalized image's values
        for z in range(no_levels):
            s = ((no_levels-1)/float(M*N))*histC[z]
            histT[z] = s
            
            imgEq[np.where(img == z)] = s
        
        return imgEq

    
    def cumulative_histogram_set(img,no_levels):        
        histC = np.zeros(no_levels).astype(int)        
        histLi = np.zeros([4,no_levels]).astype(int) # histogram for each img
        
        # generating the cumulative histogram of each img
        for i in range(4):
            histLi[i] = histogram(img,no_levels)
            histC[0] += histLi[i,0]
            for j in range(no_levels):
                histC[j] = histLi[i,j] + histC[j-1]
        
        # transfer function
        histT = np.zeros(no_levels).astype(np.uint8)
        
        # equalized image
        N,M = img.shape
        imgEq = np.zeros([N,M]).astype(np.uint8)
        
        # computing equalized image's values
        for z in range(no_levels):
            s = ((no_levels-1)/float(M*N))*histC[z]
            histT[z] = s
            
            imgEq[np.where(img == z)] = s
        
        return imgEq
    
    def gamma_correction_function(img,gamma):
        N,M = img.shape
        for x in range(N):
            for y in range(M):
                img[x,y] = math.floor(255*((img[x,y]/float(255))**(1/gamma)))
        return img
        
    L_enhan = L
    if F == 1:
        # for each low res image
        for i in range(4):
            L_enhan[i] = individual_cumulative_histogram(L[i],256)
    elif F == 2:
        for i in range(4):
            L_enhan[i] = cumulative_histogram_set(L[i],256)
    elif F == 3:
        for i in range(4):
            L_enhan[i] = gamma_correction_function(L[i],gamma)
    else: # no enhancement
        return L_enhan
    
    return L_enhan

In [13]:
def superresolution(L):
    """
    This function combines the images in the list using 
    a composition method to return a high resolution image.
    
    Parameters
    ----------
    L : list
       The list of size 4 containing the low quality images 
    """
    
    N = L[0].shape[0] # getting image low resolution side size
    H = np.zeros((N*2,N*2)) # high resolution image have double of the low resolution
    x, y = 0, 0
    
    # iterating in each cell of all low resolution images to compose the high resolution image
    for i in range(N): 
        for j in range(N):
            H[x,y] = L[0][i,j]
            H[x,y+1] = L[1][i,j]
            H[x+1,y] = L[2][i,j]
            H[x+1,y+1] = L[3][i,j]
            y += 2
        x += 2
        y = 0
    
    return H

In [14]:
def compare_to_reference(img,ref_filename):
    """
    This function compare a image to a reference through
    Root Mean Squared Error (RMSE). 
    The higher the SRE, the greater the difference between
    the images.
    
    Parameters
    ----------
    img : array
        The source image array
    ref_filename : string
        The reference image filename
    """
    # loading image reference from file as float
    reference = imageio.imread(ref_filename).astype(float) 
    
    N = reference.shape[0] # getting image high resolution

    rmse = np.sum(np.square(reference-img))
    rmse = math.sqrt(rmse / (N*N))
    #print(rmse)
    
    return round(rmse,4) # rounding to 4 decimal places

In [15]:
def enhancement_superresolution():
    """
    This function asks for some user inputs to read low resolution
    images from files, apply enhancement and superresolution methods 
    to increase resolution and, finally, compare to a high resolution
    image read from a file, printing the result.
    
    Inputs (in this order)
    ----------------------
    imglow : string
        The name base of the low resolution images
    imghigh : string
        The high resolution image filename
    F : int
        The enhancement function number that must be between 0 and 3
    gamma : int
        The parameter used in ehancement function number 3
    """
    
    # getting input parameters
    imglow = str(input().rstrip())
    imghigh = str(input().rstrip())
    F = int(input())
    gamma = float(input())
    
    L = load_images(imglow) # reading all low images
    L = enhancement(L,F,gamma) # applying the enhancement method in each image
    H = superresolution(L) # generating a high resolution image with the composition of the low image
    print(compare_to_reference(H,imghigh)) # print rmse result for comparison

In [17]:
if __name__ == "__main__":
    enhancement_superresolution()

imgs/02_low
imgs/02_high.png
2
1
24.351334493363314
24.3513


In [ ]:
"""
no enhancement needed
5.in
imgs/05_low
imgs/05_high.png
0
1.0

5.out
12.5620
"""